# The Atoms of Computation

*This demo is a companion to 1.1 - The Atoms of Computation.*

In this demo we look at the simplest and, arguably, least quantum of the gates available to us: `x` and `cx`.

First we will slightly modify the program from the last demo. In that case, a circuit was created that gave random outcomes. In this case, we just start with a blank circuit `qc`.

```python
qc = QuantumCircuit(2,2) 
```

We also create another circuit, `meas`, which simply contains the measurements required to extract an output.

```python
meas = QuantumCircuit(2,2)
meas.measure(0,0)
meas.measure(1,1)
```

The circuit we run is then the combination of these circuits, `qc+meas`

```python
execute(qc+meas,shots=1,get='memory')
```

Here we are running a circuit which immediately extracts an output, without doing anything before the measurement. Since qubits are always initialized in the state $|0\rangle$, which is the notation use to describe the state that gives the output `0` with certainty, the output of this two qubit circuit will be `'00'`.

The next modification is to look for player inputs. Specifically, the ▲ button is used to add an `x` gate on qubit 0, and the ▼ button adds an `x` to qubit 1. The effect of the `x` is to flip $|0\rangle$ to $|1\rangle$ and vice-versa. By pressing these buttons, we therefore manually change the output between the four possible outputs. A variable `old_keys` is used to make sure an `x` gate is only added when the button is pressed.

Note that Qiskit labels bits from right to left. So the output of qubit 0 will be displayed in the left square.

If you are using the emulator embedded in these notebooks, open and run the [controller notebook](controller.ipynb). This will give you a controller with which you can interact with the process on screen. Make sure to run this before any PewPew emulation processes are running.

In [ ]:
%matplotlib notebook

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits

# create a circuit with the required measurements, so we can add them in easily
meas = QuantumCircuit(2,2)
meas.measure(0,0)
meas.measure(1,1)

# loop over the squares centered on (1,2) and (6,2) and make all dim
for (X,Y) in [(1,2),(6,2)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
        
for (X,Y) in [(1,2),(6,2)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys&pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory
    m = execute(qc+meas,shots=1,get='memory')
    
    # turn the pixel (1,2) on or off depending on whether the first bit value is 1 or 0
    if m[0][0]=='1':
        screen.pixel(1,2,3)
    else:
        screen.pixel(1,2,0)
    # do the same for pixel (6,2)
    if m[0][1]=='1':
        screen.pixel(6,2,3)
    else:
        screen.pixel(6,2,0)

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

In the above we are just using qubits to store bit values, and using the `x` gate to flip those values between `'0'` and `'1'`. Now it's time to try out a little computation.

In [1.1 - The Atoms of Computation](../ch0/1.1.ipynb), we saw how to create a half adder using the `cx` and `ccx` gates. The `cx` calculated the least significant digit fo the answer, which is just the XOR of the inputs. The `ccx` calculated the most significant digit.

Since Aether only gives us two qubits to play with, and the `ccx` requires three, we won't be fully able to reproduce the half adder here. However, we can at least use the `cx` to calculate the XOR of the two bit values.

We do this by simply adding a `cx` into the circuit before measurement. Whichever qubit is the 'target' of this gate (the second qubit specified in its argument) will have its output essentially overwritten with result of the XOR.

The way we'll add the `cx` gate will be to make a circuit just containing a `cx`. 

```python
cnot = QuantumCircuit(2,2)
cnot.cx(0,1)
```

We can then add it in before the measurement when we calculate the XOR.

```python
m = execute(qc+cnot+meas,shots=1,get='memory')
xor = m[0][0]
```

Remember that Python numbers the characters in a string from left to right, whereas Qiskit labels bits in a bit string from right to left. So for the four possible output values contained in `m[0]` (`'00'`, `'01'`, `'10'` and `'11'`), it is the bit value on the left that describes the output of qubit 0 and the bit value on the right that describes that for qubit 1. This means `m[0][1]` is the output for qubit 0, and `m[0][0]` is that for qubit 1 is that for qubit 1. Since the `cx` we use has qubit 1 as the target, the XOR result will be found at `m[0][0]`.

We will then display this result on a line of pixels connecting the two squares: on for `1` and off for `0`.

```python
# display the xor on the middle pixels
if xor=='0':
    for (X,Y) in [(3,2),(4,2)]:
        screen.pixel(X,Y,0)
else:
    for (X,Y) in [(3,2),(4,2)]:
        screen.pixel(X,Y,2)
```

In [ ]:
import pew # setting up tools for the pewpew
from aether import QuantumCircuit, execute

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

# create an empty circuit with two qubits and two output bits
qc = QuantumCircuit(2,2) 

cnot = QuantumCircuit(2,2)
cnot.cx(0,1)

# create a circuit with the required measurements, so we can add them in easily
meas = QuantumCircuit(2,2)
meas.measure(0,0)
meas.measure(1,1)

# loop over the square centered on (1,2) and make all dim
for (X,Y) in [(1,2),(6,2)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
    
for (X,Y) in [(1,2),(6,2)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares       

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys&pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
    old_keys = keys 

    # execute the circuit and get a single sample of memory
    m = execute(qc+meas,shots=1,get='memory')
    
    # turn the pixel (1,2) on or off depending on whether the first bit value is 1 or 0
    if m[0][0]=='1':
        screen.pixel(1,2,3)
    else:
        screen.pixel(1,2,0)
    # do the same for pixel (6,2)
    if m[0][1]=='1':
        screen.pixel(6,2,3)
    else:
        screen.pixel(6,2,0)
        
    m = execute(qc+cnot+meas,shots=1,get='memory')
    xor = m[0][0]
    
    if xor=='0':
        for (X,Y) in [(3,2),(4,2)]:
            screen.pixel(X,Y,0)
    else:
        for (X,Y) in [(3,2),(4,2)]:
            screen.pixel(X,Y,3)
        
    pew.show(screen) # update screen to display any changes

    pew.tick(1/6) # pause for a sixth of a second  

We have now mastered all the ways you can use qubits as bits in Aether. Next we'll use qubits as qubits!

**[Click here for the next notebook](W.1.2.ipynb)**